In [1]:
# main script : sql전환 및 주석정리
import os, re


# -- re 표현식 생성 
p = re.compile('AND\s*LIMIT', re.I)
pCmt = re.compile('^\s--[\d\s\w\W]*')          #--주석만 있는라인
pCmt2 = re.compile('--[\d\s\w\W]*')            #라인내 --주석

pComp_1 = re.compile("\w*<>\w*")   # <> 연산
pComp_2 = re.compile("\w*!=\w*")    # != 연산
pComp_3 = re.compile("\w*NOT\s+IN\w*", re.I)    # NOT IN 연산
pifTest =  re.compile("\w*IF\s+TEST\w*", re.I)    # if test
pwhenTest =  re.compile("\w*WHEN\s+TEST\w*", re.I)    # when test
pNum = re.compile("999,999,999")               #to_char안에 number formatting
pselect = re.compile("\w*\(s*SELECT\s+\w*", re.I)  # (select 문
pwhen =  re.compile("\w*WHEN\w*", re.I)    # when
pnull =  re.compile("\w*IS(\s*|\s*NOT\s*)NULL\w*", re.I)  # IS NULL 또는 IS NOT NULL

def file_proc(fName):
    dName = ''
    f = open(fName,'r',encoding='utf-8')
    
    inlines = f.readlines()
    lists = []
   
    dirN, fileN = os.path.split(fName)   #디렉토리와 파일명 분리
    
    for i, line in enumerate(inlines):
        if not line: break
        line = re.sub(pCmt,'',line)    #주석만 있는라인 제거 [필요할 경우 사용]
        if len(line.strip()) == 0: continue
        line = line.strip()
       
        if re.search(pifTest,line):  # if test 포함되면 skip
            continue
        if re.search(pwhenTest,line):  # if test 포함되면 skip
            continue
            
        comp = ''
        commt = ''
        if m := re.search(pComp_1, line):
            comp = '<> 비교'
            if m1 := re.search(pwhen, line):
               if m.start() > m1.start(): 
                  commt = 'case문내' 
        elif m := re.search(pComp_2, line):
            comp = '!= 비교'
            if m1 := re.search(pwhen, line):
               if m.start() > m1.start(): 
                  commt = 'case문내' 
        elif m := re.search(pComp_3, line):
            comp = ' not in 비교'
            if m1 := re.search(pselect, line):  # not in 뒤에  select절이 나오는지 검사
                if m.start() < m1.start():
                    commt = 'select절'

        if comp:
#             print(i, comp, line)
            null_yn = ''
            if re.search(pnull, line):
                null_yn = 'null비교있음'
            lists.append(f'{dirN}`{fileN}`{str(i+1)}`{comp}`{line}`{commt}`{null_yn}')
        
    return lists

# --대상되는 파일 목록을 추출한다
sql_list = []
for root, dirs, files in os.walk('d:/Temp/공백문제/sql/'):
    for fname in files:
        full_name = os.path.join(root, fname).replace('\\','/')
        if full_name.endswith('.xml') or full_name.endswith('.txt'):
            sql_list.append(full_name) #모든 파일
 
#  각 파일에서 찾고자하는 패턴이 있는 파일정보를 취합한다
res_list = []
for fname in sql_list:
    res = file_proc(fname)
    if len(res) > 0:
        res_list.extend(res)


In [2]:
''' 리스트를 df를 전환하여 엑셀로 출력'''
import pandas as pd
cols = ['폴더','파일','라인','패턴','문장','참고','null문 유무']
df = pd.DataFrame( [i.split('`') for i in res_list], columns = cols)
df.to_excel('c:/temp/목록.xlsx', index = False)